<a href="https://colab.research.google.com/github/imkaywu/notebooks/blob/main/movie_text_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is adapted from this [tutorial](https://towardsdatascience.com/how-to-create-a-vector-based-movie-recommendation-system-b6d4f7582d66).

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# You need to put the "kaggle.json" file in a directory and assign it to "KAGGLE_CONFIG_DIR" env var.
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/kaggle"

## Preprocess movie dataset

In [3]:
!kaggle datasets download -d rounakbanik/the-movies-dataset
# !kaggle datasets download -d cryptexcode/mpst-movie-plot-synopses-with-tags

 99% 226M/228M [00:01<00:00, 228MB/s]
100% 228M/228M [00:01<00:00, 160MB/s]


In [4]:
!mkdir -p movie_dataset && unzip \*.zip -d movie_dataset && rm *.zip

Archive:  the-movies-dataset.zip
  inflating: movie_dataset/credits.csv  
  inflating: movie_dataset/keywords.csv  
  inflating: movie_dataset/links.csv  
  inflating: movie_dataset/links_small.csv  
  inflating: movie_dataset/movies_metadata.csv  
  inflating: movie_dataset/ratings.csv  
  inflating: movie_dataset/ratings_small.csv  


In [21]:
import pandas as pd

df = pd.read_csv('movie_dataset/movies_metadata.csv')

<ipython-input-21-9e46951f10b2>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movie_dataset/movies_metadata.csv')


In [22]:
df = df[~df.title.isnull()]
df = df[~df.original_title.isnull()]
df = df[~df.overview.isnull()]
df = df.sort_values('title').reset_index(drop=True)
df['lev'] = None
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
0,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.womenartrevolution.com/,55245,tt1699720,en,!Women Art Revolution,"Through intimate interviews, provocative art, ...",...,0.0,83.0,[],Released,40 years in the making!,!Women Art Revolution,False,4.3,2.0,None
1,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,41371,tt1637976,en,#1 Cheerleader Camp,A pair of horny college guys get summer jobs a...,...,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grab your spirit sticks!,#1 Cheerleader Camp,False,3.4,23.0,None
2,False,NaN,1500000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://www.hashtaghorror.com/,301325,tt3526286,de,#Horror,"Inspired by actual events, a group of 12 year ...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Death is trending.,#Horror,False,3.4,53.0,None
3,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.chicagogirlfilm.com/,267752,tt3060338,en,#chicagoGirl,From her childhood bedroom in the Chicago subu...,...,0.0,74.0,[],Released,The Social Network Takes on a Dictator,#chicagoGirl,False,7.0,1.0,None
4,False,NaN,0,"[{'id': 37, 'name': 'Western'}]",NaN,143747,tt0060697,it,Mille dollari sul nero,Johnny Liston has just been released from pris...,...,0.0,104.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,NaN,"$1,000 on the Black",False,6.0,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44501,False,NaN,5000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://www.y-film.ru/,44839,tt1372747,ru,Юленька,"A university professor, who wants to slow down...",...,0.0,102.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,Юленька,False,5.0,10.0,None
44502,False,NaN,0,[],NaN,120249,tt0130430,en,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,Described (rather cheekily) by director Michae...,...,0.0,270.0,[],Released,NaN,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,False,10.0,1.0,None
44503,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,14670,tt0090557,en,’Round Midnight,Inside the Blue Note nightclub one night in 19...,...,0.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The music and the magic come together...,’Round Midnight,False,6.7,20.0,None
44504,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,109380,tt0123368,cs,…a pátý jezdec je Strach,A Jewish doctor helps a political fugitive dur...,...,0.0,100.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Released,NaN,…And the Fifth Horseman Is Fear,False,8.0,4.0,None


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44506 entries, 0 to 44505
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  44506 non-null  object 
 1   belongs_to_collection  4431 non-null   object 
 2   budget                 44506 non-null  object 
 3   genres                 44506 non-null  object 
 4   homepage               7761 non-null   object 
 5   id                     44506 non-null  object 
 6   imdb_id                44491 non-null  object 
 7   original_language      44496 non-null  object 
 8   original_title         44506 non-null  object 
 9   overview               44506 non-null  object 
 10  popularity             44506 non-null  object 
 11  poster_path            44163 non-null  object 
 12  production_companies   44506 non-null  object 
 13  production_countries   44506 non-null  object 
 14  release_date           44435 non-null  object 
 15  re

In [8]:
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.5 MB/s eta 0:00:00


In [24]:
from Levenshtein import distance

for i in range(len(df)-1):
    at = df.iloc[i].title
    bt = df.iloc[i+1].title
    ai = df.iloc[i].imdb_id
    bi = df.iloc[i+1].imdb_id
    if distance(at, bt) <= 3 and ((ai == '0' or bi == '0') or (ai != '0' and bi != '0' and ai == bi)):
        df.at[i, 'lev'] = distance(at, bt)
        print(i, at, bt)

#we filter similar movies
df = df[df['lev'].isnull()].reset_index(drop=True)
df

819 A Farewell to Arms A Farewell to Arms
1207 A Place at the Table A Place at the Table
4953 Black Gold Black Gold
5066 Blackout Blackout
6001 Brotherhood Brotherhood
6042 Bubble Bubble
6413 Camille Claudel 1915 Camille Claudel 1915
6844 Cemetery of Splendour Cemetery of Splendour
7549 Clockstoppers Clockstoppers
7913 Confessions of a Dangerous Mind Confessions of a Dangerous Mind
8936 Days of Darkness Days of Darkness
9094 Deal Deal
12473 Force Majeure Force Majeure
19034 Le Samouraï Le Samouraï
22844 Nana, the True Key of Pleasure Nana, the True Key of Pleasure
23834 Offside Offside
25501 Pokémon 4Ever: Celebi - Voice of the Forest Pokémon 4Ever: Celebi - Voice of the Forest
25518 Pokémon: Spell of the Unknown Pokémon: Spell of the Unknown
26951 Rich and Famous Rich and Famous
28427 Seven Years Bad Luck Seven Years Bad Luck
33235 The Congress The Congress
37630 The Phantom of the Opera The Phantom of the Opera
37881 The Promise The Promise
39738 The Viking The Viking
39846 The Warri

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
0,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.womenartrevolution.com/,55245,tt1699720,en,!Women Art Revolution,"Through intimate interviews, provocative art, ...",...,0.0,83.0,[],Released,40 years in the making!,!Women Art Revolution,False,4.3,2.0,None
1,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,41371,tt1637976,en,#1 Cheerleader Camp,A pair of horny college guys get summer jobs a...,...,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grab your spirit sticks!,#1 Cheerleader Camp,False,3.4,23.0,None
2,False,NaN,1500000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://www.hashtaghorror.com/,301325,tt3526286,de,#Horror,"Inspired by actual events, a group of 12 year ...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Death is trending.,#Horror,False,3.4,53.0,None
3,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.chicagogirlfilm.com/,267752,tt3060338,en,#chicagoGirl,From her childhood bedroom in the Chicago subu...,...,0.0,74.0,[],Released,The Social Network Takes on a Dictator,#chicagoGirl,False,7.0,1.0,None
4,False,NaN,0,"[{'id': 37, 'name': 'Western'}]",NaN,143747,tt0060697,it,Mille dollari sul nero,Johnny Liston has just been released from pris...,...,0.0,104.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,NaN,"$1,000 on the Black",False,6.0,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44474,False,NaN,5000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://www.y-film.ru/,44839,tt1372747,ru,Юленька,"A university professor, who wants to slow down...",...,0.0,102.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,Юленька,False,5.0,10.0,None
44475,False,NaN,0,[],NaN,120249,tt0130430,en,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,Described (rather cheekily) by director Michae...,...,0.0,270.0,[],Released,NaN,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,False,10.0,1.0,None
44476,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,14670,tt0090557,en,’Round Midnight,Inside the Blue Note nightclub one night in 19...,...,0.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The music and the magic come together...,’Round Midnight,False,6.7,20.0,None
44477,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,109380,tt0123368,cs,…a pátý jezdec je Strach,A Jewish doctor helps a political fugitive dur...,...,0.0,100.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Released,NaN,…And the Fifth Horseman Is Fear,False,8.0,4.0,None


In [25]:
# manual check: find Avengers duplicates
for a in range(len(df)):
    if df.iloc[a]['title'].find('Avengers') != -1:
        pass
        print(a, df.iloc[a]['title'], df.iloc[a]['imdb_id'])

# drop extra
# df = df.drop([9572]).reset_index(drop=True) #i can do 1, 2, 3... to drop multiple
# df
df.to_csv('movie_dataset/movies_metadata_dedup.csv')

337 3 Avengers tt0058651
3528 Avengers Confidential: Black Widow & Punisher tt3482378
3529 Avengers Grimm tt4296026
3530 Avengers: Age of Ultron tt2395427
8301 Crippled Avengers tt0077292
21023 Masked Avengers tt0082153
23095 Next Avengers: Heroes of Tomorrow tt1259998
32065 The Avengers tt0848228
32066 The Avengers tt0118661
37143 The New Adventures of the Elusive Avengers tt0063369
38580 The Shaolin Avengers tt0074513
41771 Ultimate Avengers tt0491703
41772 Ultimate Avengers 2 tt0803093


## Encode data

In [10]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e40c0e4c1c2e8534c90b790e7f2477b70061aee87a0b41132a07c2acc81f39fa
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [27]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2')

df_ = df.copy()
df_['desc'] = df_['original_title'] + ", " + df_['title'] + ': ' + df_['overview']
df_['desc'] = df_['desc'].progress_apply(lambda x : model.encode(x))
df_index = df_.pop('title')
df_ = df_[['desc']]
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_.index = df_index
df_

100%|██████████| 44479/44479 [06:15<00:00, 118.30it/s]


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,0.025099,0.029374,-0.027960,0.014324,0.060275,0.103648,-0.063354,-0.090073,-0.005252,-0.081225,...,0.046094,0.046670,0.026144,0.051195,-0.027308,0.082337,0.092032,0.006015,0.047258,-0.087466
#1 Cheerleader Camp,0.002672,-0.078156,0.064948,0.055613,-0.000306,-0.010688,0.144935,-0.070611,-0.019296,-0.004697,...,0.009700,0.025112,-0.038107,0.038656,0.087143,-0.001401,0.024417,-0.014034,-0.019905,-0.032767
#Horror,0.003017,0.026786,-0.039990,0.001537,0.037760,0.020078,0.065459,0.002187,0.057069,-0.051378,...,0.151221,-0.080794,0.013428,0.052292,-0.015831,0.116267,0.055043,0.023725,-0.025115,0.008064
#chicagoGirl,0.045128,0.034480,-0.021918,-0.117925,-0.010988,0.048915,0.038719,-0.019365,-0.015623,0.086167,...,0.060104,-0.040004,0.057313,0.031353,0.007605,0.024501,0.055037,-0.038720,-0.081106,-0.048068
"$1,000 on the Black",-0.127467,0.064530,-0.102074,0.070310,-0.012053,0.039369,0.056251,0.057213,-0.012269,-0.077662,...,0.076908,-0.117722,0.052628,0.105615,0.025775,-0.044096,-0.010680,-0.071870,-0.008970,-0.042987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Юленька,0.007429,0.032108,-0.007530,0.009852,-0.065982,0.027610,0.028021,-0.018031,-0.001481,-0.030139,...,0.023362,-0.018582,-0.014124,-0.015888,-0.096235,-0.007553,-0.051546,0.007510,0.006350,0.003216
‘Rameau’s Nephew’ by Diderot (Thanx to Dennis Young) by Wilma Schoen,-0.034953,-0.022625,-0.027221,-0.044887,0.003829,0.110865,0.045596,0.069290,0.069122,-0.079512,...,0.058469,-0.043333,-0.038951,0.118606,-0.066927,0.090684,0.056830,0.068711,0.041658,-0.026577
’Round Midnight,-0.005495,-0.020401,-0.016184,-0.010237,-0.078786,0.034183,0.067951,0.019022,0.051592,-0.057580,...,0.014334,-0.033659,-0.022284,0.109464,-0.017973,-0.027031,-0.013382,0.087385,-0.070542,-0.061830


In [30]:
df_.to_csv('movie_dataset/movies_metadata_dedup_encoded.csv')

## Perform nearest neighbor search

In [31]:
df_movies_encoded = pd.read_csv('movie_dataset/movies_metadata_dedup_encoded.csv')
df_movies_encoded.index = df_movies_encoded.pop('title')
df_movies_encoded

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,0.025099,0.029374,-0.027960,0.014324,0.060275,0.103648,-0.063354,-0.090073,-0.005252,-0.081225,...,0.046094,0.046670,0.026144,0.051195,-0.027308,0.082337,0.092032,0.006015,0.047258,-0.087466
#1 Cheerleader Camp,0.002672,-0.078156,0.064948,0.055613,-0.000306,-0.010688,0.144935,-0.070611,-0.019296,-0.004697,...,0.009700,0.025112,-0.038107,0.038656,0.087143,-0.001401,0.024417,-0.014034,-0.019905,-0.032767
#Horror,0.003017,0.026786,-0.039990,0.001537,0.037760,0.020078,0.065459,0.002187,0.057069,-0.051378,...,0.151221,-0.080794,0.013428,0.052292,-0.015831,0.116267,0.055043,0.023725,-0.025115,0.008064
#chicagoGirl,0.045128,0.034480,-0.021918,-0.117925,-0.010988,0.048915,0.038719,-0.019365,-0.015623,0.086167,...,0.060104,-0.040004,0.057313,0.031353,0.007605,0.024501,0.055037,-0.038720,-0.081106,-0.048068
"$1,000 on the Black",-0.127467,0.064530,-0.102074,0.070310,-0.012053,0.039369,0.056251,0.057213,-0.012269,-0.077662,...,0.076908,-0.117722,0.052628,0.105615,0.025775,-0.044096,-0.010680,-0.071870,-0.008970,-0.042987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Юленька,0.007429,0.032108,-0.007530,0.009852,-0.065982,0.027610,0.028021,-0.018031,-0.001481,-0.030139,...,0.023362,-0.018582,-0.014124,-0.015888,-0.096235,-0.007553,-0.051546,0.007510,0.006350,0.003216
‘Rameau’s Nephew’ by Diderot (Thanx to Dennis Young) by Wilma Schoen,-0.034953,-0.022625,-0.027221,-0.044887,0.003829,0.110865,0.045596,0.069290,0.069122,-0.079512,...,0.058469,-0.043333,-0.038951,0.118606,-0.066927,0.090684,0.056830,0.068711,0.041658,-0.026577
’Round Midnight,-0.005495,-0.020401,-0.016184,-0.010237,-0.078786,0.034183,0.067951,0.019022,0.051592,-0.057580,...,0.014334,-0.033659,-0.022284,0.109464,-0.017973,-0.027031,-0.013382,0.087385,-0.070542,-0.061830


### Method 1

**Deprecated**

In [15]:
from Levenshtein import distance
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df_movies_encoded)
#string-searching algorithm
def closest_title(title):
    m = pd.DataFrame(df_movies_encoded.index)
    m['lev'] = m['title'].apply(lambda x : distance(x, title))
    return m.sort_values('lev', ascending=True)['title'].iloc[0]

In [16]:
def find_similar_movies(df, nbrs, title):
    #if title not in df it will choose the best search
    title = closest_title(title)

    distances, indices = nbrs.kneighbors([df.loc[title]])
    #print(indices)
    #we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

In [17]:
find_similar_movies(df_movies_encoded, nbrs, 'Prince of Egypt')

index 11440
The Prince of Egypt -> The Ten Commandments: The Musical


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


### Method 2: Euclidean distance

In [14]:
embeddings = df_movies_encoded.to_numpy(dtype='float32')
embeddings.dtype

dtype('float32')

In [65]:
def calc_pairwise_dist(features1, features2):
  N, D = features1.shape
  T, _ = features2.shape
  assert features1.dtype == features2.dtype
  dtype = features1.dtype
  return np.matmul(features1**2, np.ones((D, T), dtype=dtype)) + np.matmul(np.ones((N, D), dtype=dtype), features2.T**2) - 2 * np.matmul(features1, features2.T)

# a = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]], dtype=int)
# b = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]], dtype=int)
# calc_pairwise_dist(a, b)

In [68]:
def find_similar_videos_0(query, top_k=3):
  query_embedding = model.encode(query)

  dist = calc_pairwise_dist(embeddings, query_embedding[None, :])
  dist = np.squeeze(dist, axis=1)
  indices = np.argsort(dist)
  return df.iloc[indices[:top_k]]

In [66]:
import torch

embeddings_t = torch.from_numpy(embeddings)

def find_similar_videos_1(query, top_k=3):
  query_embedding = model.encode(query)
  query_embedding_t = torch.from_numpy(query_embedding)[None, :]
  res = util.semantic_search(query_embedding_t, embeddings_t, top_k=top_k)
  indices = [item['corpus_id'] for item in res[0]]
  return df.iloc[indices[:top_k]]

In [81]:
query = '''The Avengers follows the aftermath of several previous Marvel Cinematic Universe (MCU) films, where various superheroes have been introduced, each with their own unique abilities and backgrounds. The movie opens with the Tesseract, a powerful energy source, falling into the hands of the malevolent Loki (Tom Hiddleston), who plans to use it to conquer Earth and subjugate humanity. In response to the imminent threat, Nick Fury (Samuel L. Jackson), the director of S.H.I.E.L.D., assembles a team of extraordinary individuals to thwart Loki’s nefarious plans. This team, known as the Avengers, comprises Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye. However, their initial inability to work together as a cohesive unit threatens to jeopardize their mission. As the Avengers confront Loki and his army of Chitauri invaders in New York City, they must overcome their differences and egos to forge a formidable alliance. Together, they showcase their unique strengths and talents in a pulse-pounding battle to protect Earth and its inhabitants from ultimate destruction.'''

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
32065,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,...,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0,None
3530,False,"{'id': 86311, 'name': 'The Avengers Collection...",280000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://marvel.com/movies/movie/193/avengers_ag...,99861,tt2395427,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,...,1.405404e+09,141.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A New Age Has Come.,Avengers: Age of Ultron,False,7.3,6908.0,None
6504,False,"{'id': 131295, 'name': 'Captain America Collec...",250000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://marvel.com/captainamericapremiere,271110,tt3498820,en,Captain America: Civil War,"Following the events of Age of Ultron, the col...",...,1.153304e+09,147.0,"[{'iso_639_1': 'ro', 'name': 'Română'}, {'iso_...",Released,Divided We Fall,Captain America: Civil War,False,7.1,7462.0,None


In [82]:
query = 'chris evans #chrisevans #paulrudd #captainamerica #antman #theavengers #avengers #marvel #sebastianstan'

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
32065,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,...,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0,None
31376,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,413279,tt6016776,en,Team Thor,Discover what Thor was up to during the events...,...,0.000000e+00,3.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Team Thor,False,7.5,93.0,None
6506,False,"{'id': 131295, 'name': 'Captain America Collec...",170000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.captainamericathewintersoldiermovie...,100402,tt1843866,en,Captain America: The Winter Soldier,After the cataclysmic events in New York with ...,...,7.147666e+08,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In heroes we trust.,Captain America: The Winter Soldier,False,7.6,5881.0,None


In [83]:
query = 'Ayo this thing racist #spiderman #avengers #marvel #marvelstudios #funny #foryou'

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
29895,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...",NaN,49131,tt0499665,en,Spiderman: The Ultimate Villain Showdown,Spider-Man meets some of his greatest foes inc...,...,0.000000e+00,79.0,[],Released,Four episodes of the popular 90's animated ser...,Spiderman: The Ultimate Villain Showdown,False,5.2,3.0,None
32065,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,...,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0,None
30821,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",NaN,43641,tt1701223,en,Superman/Shazam!: The Return of Black Adam,Chosen the world’s protector against the Seven...,...,0.000000e+00,25.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We fight for those you can't fight for themsel...,Superman/Shazam!: The Return of Black Adam,False,7.1,77.0,None


In [22]:
df.iloc[30821].overview

'Chosen the world’s protector against the Seven Deadly Enemies of Man – pride, envy, greed, hatred, selfishness, laziness and injustice – young Billy Batson accepts his destiny as Captain Marvel. Battling alongside Superman against nefarious Black Adam, Billy soon discovers the challenge super heroes ultimately face: is it revenge or justice?'

In [84]:
query = 'It focuses on the transformation of his youngest son, Michael Corleone, from reluctant family outsider to ruthless mafia boss.'

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
34729,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,245066411.0,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,None
34730,False,NaN,0,[],NaN,364150,tt0150742,en,The Godfather Trilogy: 1972-1990,The multigenerational saga of the rise and fal...,...,0.0,583.0,[],Released,NaN,The Godfather Trilogy: 1972-1990,True,8.8,13.0,None
34731,False,"{'id': 230, 'name': 'The Godfather Collection'...",13000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,240,tt0071562,en,The Godfather: Part II,In the continuing saga of the Corleone crime f...,...,47542841.0,200.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"I don't feel I have to wipe everybody out, Tom...",The Godfather: Part II,False,8.3,3418.0,None


In [85]:
query = '''This is Quentin Tarantino at his finest! Engaging characters; non-stop action; and intricately woven individual stories that produce a mesmerizing whole.

The depth of the overall plot is astounding. One minute dark and moody, the next riddled with black comedy and unforgettable utterances that remain noteworthy
quotes even into present day. Violence and blood aplenty, rampant drug use, startlingly intense situations- all signature Tarantino. And yet, this film is interspersed with sparkling nuggets of stark morality and unexpected wisdom. John Travolta as Vincent Vega, Samuel L. Jackson as Jules, Uma Thurman as Mia Wallace, and Bruce Willis as Butch are exceptionally complex characters who must learn to survive in an often cold and brutal world that still has hidden gems of humanity at its most basic fineness. Pulp Fiction is a roller coaster ride that will leave you gasping for more!'''

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
26021,False,NaN,8000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,680,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",...,213928762.0,154.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670.0,None
13499,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,266880,tt3661072,en,Giuseppe Makes a Movie,"While the rest of America slept, DIY filmmaker...",...,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,WARNING: THIS FILM CONTAINS SENIOR CITIZEN NUD...,Giuseppe Makes a Movie,False,6.4,4.0,None
11951,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,311998,tt0876257,en,Film Noir: Bringing Darkness to Light,Film Noir burrows into the mind; it's disorien...,...,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Film Noir: Bringing Darkness to Light,False,7.3,4.0,None


In [86]:
query = "I haven't seen The Godfather in years but as it was on BBC2 last night, 13 December, decided to watch it again.  So glad I did as it is a superb film.  Although made in 1971-72 it has not dated, as so many films of that decade do.  Performances are superb, notably Marlon Brando and Al Pacino.  Pacino's face is like a mask - no emotion is shown  and he can casually order an opponent's execution as sitting down to a meal.  I also love the stillness of the movie and how family life was vividly shown - the Italian love of good food and drink.  Colours are muted in browns and golds and Coppola made a masterpiece.  5 star rating."

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
34729,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,245066411.0,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,None
34730,False,NaN,0,[],NaN,364150,tt0150742,en,The Godfather Trilogy: 1972-1990,The multigenerational saga of the rise and fal...,...,0.0,583.0,[],Released,NaN,The Godfather Trilogy: 1972-1990,True,8.8,13.0,None
6953,False,"{'id': 296358, 'name': 'Anche gli angeli - Col...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,41897,tt0071139,it,Anche gli angeli tirano di destro,This film has the typical air of Italian actio...,...,0.0,111.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Charleston,False,0.5,1.0,None


In [26]:
df.iloc[6953].overview

'This film has the typical air of Italian action/gangster comedies from that period - it is a perfectly charming and funny sort of comedy. The cliches of black and white are wonderfully maintained in this movie as the good guys are clever and witty and the bad ones are bad and pretty dumb. The "Shakespearian" dream of the "good" Mafioso a bit stupid but then a film like that seems in need of a few blunders.'

In [87]:
query = "复仇者联盟"

find_similar_videos_0(query, 3)
# print(find_similar_videos_1(query, 3).title)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,lev
23570,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,36210,tt0209189,zh,一個都不能少,Teacher Gao loves the students in his poor vil...,...,0.0,106.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,"In her village, she was the teacher. In the ci...",Not One Less,False,7.5,36.0,None
25230,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",http://www.pigsthemovie.com/,18755,tt1065106,en,Pigs,A college ladies man accepts a challenge from ...,...,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,26 girls. 26 letters in the alphabet. 1 campus...,Pigs,False,4.3,22.0,None
3841,False,NaN,14000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",NaN,348892,tt3863552,hi,Bajrangi Bhaijaan,A young mute girl from Pakistan loses herself ...,...,98000000.0,163.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}, {'iso_...",Released,NaN,Bajrangi Bhaijaan,False,7.7,165.0,None
